In [ ]:
'''
[주의] 
!!!! 아직 dropout_data 만드는 중! 어디까지 만들었는지 확인하고 코드 돌리세요 존재 안하는 폴더 있을수도 !!!!
!!!!!dropout_data의 X는 sparse array, int32으로 통일했습니다.!!!!!!
!!!! adata 모두 normalize/log 했습니다. 전에 없던 오류가 생겼으면 얘가 원인일수도... !!!
봐서 알겠지만 원본/dropout 파일들의 이름이 같습니다. 코딩하기 편하려고 통일했으니
아무쪼록 헷갈리지 마시길...
dropout_data에 layers/dropped가 추가되어있습니다. X랑 같은 모양으로 boolean 
형식이고, True인 데이터들이 drop 후 0으로 채운 위치입니다. 혹시 모델이 NaN을
더 잘 impute한다면 이거 이용해서 데이터 수정하시면 됩니다.
모델 별로 이 노트북을 새로 복제해서 만들어두면 나중에 배포용 코드 만들기 편하겠죠?
이미지 파일이 필요하시면 hest_data/wsis 에서 가져오셔야 합니다. SPROD에 제가 만들어둔 가져오는
코드가 있긴 하니 필요하시면 가져가시죠.
이왕이면 dropout_data 폴더는 수정하지 말아주세요 (output 저장 포함)

결과는 bench_results에 저장됩니다. 혹시 잘못 돌려서 데이터 삭제해야하면 수동으로 들어가서 수정하시죠 ㅎ

[수정] 이라고 코멘트 해 둔 부분 적절히 고치시면 됩니다.
'''

In [ ]:
import os
import GPUtil

In [ ]:
# Determine the best GPU for stlearn/TensorFlow with the priority: cuda:2, cuda:0, cuda:1
# If none is available, print message and use CPU for stlearn.

device_priority = [1,0, 2]  # Highest priority first

# Get the list of available GPUs
available_gpus = GPUtil.getAvailable(order='first', limit=len(device_priority), maxLoad=0.5, maxMemory=0.5, includeNan=False)

# Select GPUs based on priority
selected_tf_devices = []
for d in device_priority:
    if d in available_gpus:
        selected_tf_devices.append(str(d))
        # Uncomment the next line if you want to select only one GPU based on priority
        # break

# Set CUDA_VISIBLE_DEVICES
if not selected_tf_devices:
    print("No suitable GPU found for stlearn/TensorFlow. Using CPU for stSME.")
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
else:
    os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(selected_tf_devices)
    print(f"Using GPU(s) {', '.join(selected_tf_devices)} for stlearn/TensorFlow.")


In [ ]:
import torch
import tensorflow as tf

device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu') #[수정] cuda:사용할 GPU
print(f"Using device {device} for PyTorch operations.")

In [ ]:
import anndata as ad
import numpy as np
from sklearn.metrics import adjusted_rand_score
from scipy.sparse import csr_matrix
import scanpy as sc
import pandas as pd

In [ ]:
# import modules
import stlearn as st
from pathlib import Path
from tifffile import TiffFile
from parallel_tilingv2_batch import tiling_optimized
from parallel_extract_feature import extract_feature_optimized
import random

st.settings.set_figure_params(dpi=180)

In [ ]:
bench_data_dir = '/home/s2022310812/bench_data' # bench_data 경로 [수정]
dropout_data_dir = '/home/s2022310812/dropout_data' #dropout_data 경로 [수정]
datasets = [ #[수정]
    "IDC",
  # "COAD",
  # "READ",
  # "LYMPH_IDC",
]
model = "stSME" #[수정] 모델명
percents = [ #[수정] dropout percentage
    '30%',
    #'20%',
    #'30%'
] 
samples = [ #[수정] dropout percentage data version
    '1',
    #'2',
    #'3',
    #'4',
    #'5'
] 

trial_range = range(1, 6)  # 5 trials

man_sc = {}

# Option to save results to CSV or just print them out
save_results = True  


In [ ]:
# Image directory for tiling
wsis_dir = '/home/s2022310812/hest_data/wsis'  # The directory where WSIs are stored

TILE_PATH = Path("/home/s2022310812/stlearn/stSME/bench/v1/tmp/tiles")
TILE_PATH.mkdir(parents=True, exist_ok=True)

In [ ]:
for trial in trial_range[:1]:
    seed = trial
    print(f"\n===== Starting Trial {trial} with seed {seed} =====")
    
    # -------------------- Set Random Seeds --------------------
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    # Set TensorFlow's random seed
    tf.random.set_seed(seed)
    
    print(f"Random seed set to {seed} for Trial {trial}.")

    for dataset in datasets:
        for percent in percents:
            for sample in samples:
                bench_path = os.path.join(bench_data_dir,dataset,'adata')
                dropout_path = os.path.join(dropout_data_dir,dataset,percent+"-"+sample)
                
                adata_names = [f for f in os.listdir(bench_path) if os.path.isfile(os.path.join(bench_path, f))]
            
                for adata_name in adata_names[:3]:
    
                    adata = ad.read_h5ad(os.path.join(bench_path,adata_name))
                    sc.pp.normalize_total(adata, target_sum=1e6, exclude_highly_expressed= False) 
                    sc.pp.log1p(adata)
                    sc.pp.scale(adata)
                    
                    adata_drop = ad.read_h5ad(os.path.join(dropout_path,adata_name))
                    sc.pp.normalize_total(adata_drop, target_sum=1e6, exclude_highly_expressed= False)
                    sc.pp.log1p(adata_drop)
                    sc.pp.scale(adata_drop)
                    dropped_mask = torch.tensor(adata_drop.layers['dropped'], dtype=torch.bool, device=device)
    
                    """
                    [수정]
                    여기서부터 [여기까지] adata_drop.X에 본인 모델 적용시켜서 adata2에 저장해주시면 됩니다.
                    [주의] adata2에 ""adata.X 에 해당하는 파일"" 저장해주세요... 그냥 그렇게 만들었어요..
                    예시로 제 코드 보여드림. 여러분한테는 쓸모가 없겠죠? 지우고 시작하시면 됩니다 화이팅
                    """
                   
                    # Derive base_name from adata_name (remove .h5ad extension)
                    base_name = adata_name.replace(".h5ad", "")
    
                    # Use base_name as sample_id for clarity (optional but recommended)
                    sample_id = base_name
    
                    # Load the corresponding .tif image
                    tif_file = os.path.join(wsis_dir, f"{base_name}.tif")
    
                    if os.path.exists(tif_file):
                        # Load the .tif file and extract the highest resolution image
                        with TiffFile(tif_file) as tif:
                            highest_res_image = tif.pages[0].asarray()
    
                        # Check if the image is not all zeros
                        if not np.any(highest_res_image):
                            print(f"Warning: The loaded image {tif_file} contains only zeros.")
                        else:
                            print(f"Loaded image {tif_file} with shape {highest_res_image.shape}.")
                            
                        # Normalize image [0,255] -> [0,1] if needed
                        highest_res_image = highest_res_image.astype(float) / 255.0
    
                        # Construct the spatial dictionary under adata_drop.uns
                        adata_drop.uns["spatial"] = {
                            sample_id: {
                                "images": {
                                    "hires": highest_res_image
                                },
                                "scalefactors": {
                                    "tissue_hires_scalef": 1
                                },
                                "use_quality": "hires"
                            }
                        }
    
                        # Ensure imagerow and imagecol columns in adata_drop.obs
                        # Assuming adata_drop.obsm["spatial"] is Nx2 with [x,y] in pixel coordinates.
                        adata_drop.obs["imagerow"] = adata_drop.obsm["spatial"][:, 1]  # y-coordinate
                        adata_drop.obs["imagecol"] = adata_drop.obsm["spatial"][:, 0]  # x-coordinate
    
                        # Check the structure
                        print("adata_drop.uns['spatial'] structure:")
                        print(adata_drop.uns["spatial"])
                    else:
                        print(f"TIF file {tif_file} does not exist. Please check your wsis_dir or file name.")
    
                    # Apply stSME
                    tiling_optimized(adata_drop, out_path=TILE_PATH, crop_size=40, target_size=299, num_workers=40, verbose=True)
                    #st.pp.tiling(adata_drop, out_path=TILE_PATH, crop_size=40, target_size=299)
                    #extract_feature_optimized(adata_drop, cnn_base="resnet50", n_components=50, seeds=seed)
                    st.pp.extract_feature(adata_drop, cnn_base="resnet50", n_components=50, seeds=seed)
    
                    # Run PCA before SME
                    st.em.run_pca(adata_drop, n_comps=50)
    
                    # SME normalization using data already normalized/log-transformed:
                    st.spatial.SME.SME_normalize(adata_drop, use_data="raw")
                    
                    # After normalization, SME results are in adata_drop.obsm['raw_SME_normalized']
                    # After normalization, SME results are in adata_drop.obsm['raw_SME_normalized']
                    if "raw_SME_normalized" not in adata_drop.obsm:
                        print(f"Error: SME normalization did not add 'raw_SME_normalized' to adata.obsm for {adata_name}.")
                        continue
    
                    # Save SME results to adata2
                    adata2 = adata_drop.obsm['raw_SME_normalized']
                    
                    """
                    [여기까지]
                    """
                    
                    
                    
                    # Step 1: Convert adata.X to float32 or int32 before creating the PyTorch tensor
                    if isinstance(adata.X, np.ndarray):
                        # Convert to float32 if data is uint16 or other unsupported types
                        adata_X = adata.X.astype(np.float32) if adata.X.dtype == np.uint16 else adata.X.astype(np.float32)
                        adata_tensor = torch.tensor(adata_X, dtype=torch.float32, device=device)
                    elif isinstance(adata.X, csr_matrix):
                        adata_X = adata.X.toarray().astype(np.float32)
                        adata_tensor = torch.tensor(adata_X, dtype=torch.float32, device=device)
                    else:
                        print('adata.X 데이터 형식 확인해주세요')
                        
                    # Step 2: Convert adata2 to float32 or int32 before creating the PyTorch tensor
                    if isinstance(adata2, np.ndarray):
                        # Convert to float32 if data is uint16 or other unsupported types
                        adata2 = adata2.astype(np.float32) if adata2.dtype == np.uint16 else adata2.astype(np.float32)
                        adata2_tensor = torch.tensor(adata2, dtype=torch.float32, device=device)
                    elif isinstance(adata2, csr_matrix):
                        adata2 = adata2.toarray().astype(np.float32)
                        adata2_tensor = torch.tensor(adata2, dtype=torch.float32, device=device)
                    else:
                        print('adata2 데이터 형식 확인해주세요')
                
                    # Step 3: Get the indices of the dropped values (where the mask is True)
                    dropped_indices = dropped_mask.nonzero(as_tuple=True)
                    
                    # Step 4: Extract the corresponding values for dropped spots
                    adata_values = adata_tensor[dropped_indices]
                    adata2_values = adata2_tensor[dropped_indices]
                    
                    # Step 5: Convert to CPU and calculate manh.
                    adata_values_cpu = adata_values.cpu().numpy()
                    adata2_values_cpu = adata2_values.cpu().numpy()
                    #print(adata_values_cpu[:10])
                    #print(adata2_values_cpu[:10])
                         
                    manhattan_distance = np.abs(adata_values_cpu - adata2_values_cpu).sum()
                    num_values = adata_values_cpu.size
                    manhattan_score = manhattan_distance / num_values
                    
                    # Store the result in the dictionary
                    man_sc[dataset + "/" + adata_name] = manhattan_score
                    
                    print(f"Processed {adata_name}, Manhattan score: {manhattan_score}")

                # After processing all adata_names
                if not man_sc:
                    print(f"No results for {dataset}/{percent}-{sample} in Trial {trial}, skipping CSV write.")
                    continue
    
                row_name = f"Trial : {trial}"
                my_dir = os.path.join('/home/s2022310812/bench_results', model,dataset)
                csv_path = os.path.join(my_dir, percent+"-"+sample +"_result.csv")
                # Ensure the directory exists
                os.makedirs(my_dir, exist_ok=True)
                
                # Extract column names and values
                columns = [key.split("/")[-1].split(".")[0] for key in man_sc.keys()]
                values = list(man_sc.values())
                
                # Create a DataFrame for the new row
                data = pd.DataFrame([values], columns=columns, index=[row_name])

                 # Only save if save_results = True
                if save_results:
                    # Attempt to read existing data
                    existing_data = None
                    if os.path.exists(csv_path):
                        print(f"CSV file '{csv_path}' exists. Attempting to read it.")
                        try:
                            existing_data = pd.read_csv(csv_path, index_col=0)
                            print(f"Successfully read existing CSV with shape {existing_data.shape}.")
                        except Exception as e:
                            print(f"Warning: Error reading existing CSV '{csv_path}': {e}")
                            print("A new CSV will be created.")
                            existing_data = None
                    else:
                        print(f"CSV file '{csv_path}' does not exist. A new one will be created.")

                    if existing_data is not None:
                        updated_data = pd.concat([existing_data, data])
                    else:
                        updated_data = data

                    # Try saving the CSV
                    print(f"Attempting to save results to '{csv_path}'...")
                    try:
                        updated_data.to_csv(csv_path)
                        print(f"Saved results to '{csv_path}'")
                    except Exception as e:
                        print(f"Error saving CSV '{csv_path}': {e}")
                else:
                    print("save_results=False, not saving results to CSV. Results for this trial:")
                    print(data)

    print(f"===== Completed Trial {trial} =====\n")

print("Benchmarking completed.")
                